In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import arviz as az

# Load processed data
df = pd.read_csv('../data/processed/log_returns.csv', parse_dates=['Date'])
prices = df['Price'].values  # Or use log_returns if preferred
dates = df['Date'].values

In [ ]:
with pm.Model() as bcp_model:
    # Priors
    tau = pm.DiscreteUniform('tau', lower=0, upper=len(prices)-1)  # Change point location
    mu1 = pm.Normal('mu1', mu=0, sigma=10)  # Mean before change
    mu2 = pm.Normal('mu2', mu=0, sigma=10)  # Mean after change
    sigma = pm.HalfNormal('sigma', sigma=10)  # Volatility
    
    # Switchpoint logic
    mu = pm.math.switch(tau >= np.arange(len(prices)), mu1, mu2)
    
    # Likelihood
    likelihood = pm.Normal('likelihood', mu=mu, sigma=sigma, observed=prices)
    
    # Sampling
    trace = pm.sample(3000, tune=1000, chains=2, target_accept=0.9)

In [ ]:
# Trace plot
pm.plot_trace(trace, var_names=['tau', 'mu1', 'mu2', 'sigma'])
plt.tight_layout()

# Summary statistics
summary = pm.summary(trace)
print(summary)

# Check R-hat (convergence)
assert all(summary['r_hat'] < 1.05), "Model did not converge!"

In [ ]:
# Extract most probable change point
tau_samples = trace['tau']
tau_est = np.argmax(np.bincount(tau_samples))  # Mode of posterior
change_date = dates[tau_est]

# Plot change point
plt.figure(figsize=(14, 6))
plt.plot(dates, prices, label='Brent Price')
plt.axvline(change_date, color='red', linestyle='--', 
            label=f'Change Point: {change_date.strftime("%Y-%m-%d")}')
plt.title('Detected Change Point in Oil Prices', fontsize=16)
plt.legend()
plt.show()

In [ ]:
# Load events
events = pd.read_csv('../data/raw/events.csv', parse_dates=['Date'])

# Find nearest event to change point
time_deltas = np.abs(events['Date'] - change_date)
closest_event = events.iloc[np.argmin(time_deltas)]

print(f"Closest Event: {closest_event['Event']} ({closest_event['Date'].strftime('%Y-%m-%d')})")
print(f"Time Delta: {np.min(time_deltas).days} days")

# Plot events near change points
plt.figure(figsize=(14, 6))
plt.plot(dates, prices, alpha=0.5)
for _, event in events.iterrows():
    plt.axvline(event['Date'], color='gray', linestyle=':', alpha=0.7)
plt.axvline(change_date, color='red', linestyle='--', linewidth=2)
plt.title('Change Points vs. Researched Events', fontsize=16)
plt.show()

In [ ]:
# Calculate mean shift
mu1_post = trace['mu1'].mean()
mu2_post = trace['mu2'].mean()
mean_shift = (mu2_post - mu1_post) / mu1_post * 100

print(f"Mean Price Before: {mu1_post:.2f}")
print(f"Mean Price After: {mu2_post:.2f}")
print(f"Change: {mean_shift:.1f}%")

# Posterior distributions
pm.plot_posterior(trace, var_names=['mu1', 'mu2'], 
                  figsize=(12, 4), hdi_prob=0.95)
plt.show()

In [ ]:
# Save change points and parameters
change_points = pd.DataFrame({
    'change_date': [change_date],
    'tau': [tau_est],
    'mu1': [mu1_post],
    'mu2': [mu2_post],
    'sigma': [trace['sigma'].mean()],
    'closest_event': [closest_event['Event']],
    'time_delta_days': [np.min(time_deltas).days]
})

change_points.to_csv('../data/processed/change_points.csv', index=False)